### Importing dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os
import random

import tensorflow as tf
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import Model
from keras.layers import Dense,Dropout,Flatten
from keras import callbacks

### Loading dataset

In [ ]:
def load_images_and_labels(data_folder):
        images=[]
        labels=[]
        num_label=[]
        img_label =0
        for label in os.listdir(data_folder):
            label_path = os.path.join(data_folder,label)

            if os.path.isdir(label_path):
                 for img in os.listdir(label_path):
                        image_path = os.path.join(label_path,img)
                        image = cv2.imread(image_path) 
                        if image is not None:
                            image = cv2.resize(image,(128,128))
                            images.append(image)
                            labels.append(label)
                            num_label.append(img_label)
                
                 img_label =img_label+1

        return images,labels,num_label

In [ ]:
data_folder = '/kaggle/input/yoga-pose-image-classification-dataset/dataset'

In [ ]:
images,labels,num_labels = load_images_and_labels(data_folder)

In [ ]:
df = pd.DataFrame(list(zip(images,labels,num_labels)),columns=['images','labels','num_label'])

In [ ]:
df.head()

### Visualizing data

In [ ]:
columns=4
rows=2

plt.figure(figsize=(10,5))
for i in range(columns*rows):
    plt.subplot(2,4,i+1)
    j=np.random.choice(range(5000))
    file =df.images[j]
    img_bgr = cv2.cvtColor(file, cv2.COLOR_RGB2BGR)
    plt.imshow(img_bgr)
    plt.title(df.labels[j])
    plt.axis('off')

### Data Augmentation

In [ ]:
X =np.array(images)/255
y= to_categorical(num_labels,num_classes=107)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0, stratify= df['num_label'])

In [ ]:
train_datagenerator = ImageDataGenerator(
    rotation_range= 20,
    shear_range =0.2,
    zoom_range =0.2,
    horizontal_flip= True,
    fill_mode= 'nearest'
    )

train_data = train_datagenerator.flow(X_train,y_train,batch_size=32)

### Model Architecture

In [ ]:
model= tf.keras.applications.DenseNet121(
                include_top=False,
                weights="imagenet",
                input_shape=(128,128,3),
                pooling= max)

model.trainable =False

In [ ]:
inputs =model.input
layer = Flatten()(model.output)

dense_layer1 = Dense(256,activation='relu')(layer)
drop_layer1 = Dropout(0.2)(dense_layer1)
dense_layer2 = Dense(128,activation='relu')(dense_layer1)
drop_layer2 = Dropout(0.2)(dense_layer2)
outputs = Dense(107,activation='softmax')(drop_layer2)

final_model = Model(inputs = inputs,outputs=outputs)

### Model training

In [ ]:
final_model.compile(optimizer ='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history =final_model.fit(train_data,epochs=25,validation_data=(X_test,y_test),steps_per_epoch=len(X_train) // 32)

### Plotting loss graph

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'],label=['Training loss'])
plt.plot(history.history['val_loss'],label=['Validation loss'])
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation loss')
plt.show()

### Plotting accuracy graph

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(history.history['accuracy'],label=['Training accuracy'])
plt.plot(history.history['val_accuracy'],label=['Validation accuracy'])
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.title('Training and Validation accuracy')
plt.show()

### Comparison with different models

In [ ]:
result = pd.DataFrame({'Model':['DenseNet121','MobileNetV2','VGG16','Inceptionv3','Resnet50','EfficientNetB3'],'Training accuracy':['0.52','0.65','0.62','0.29','0.20','0.15'],
                      'Validation accuracy':['0.47','0.51','0.59','0.32','0.23','0.15']})

In [ ]:
result